In [1]:
#------------------------------------------------------------------------------+
#
#   Samar shaaban Haytamy
#   PSO service to select optimal web service  with Python
#   OCT, 2018
#
#------------------------------------------------------------------------------+

#--- IMPORT DEPENDENCIES ------------------------------------------------------+

from __future__ import division
import random
import time

from pandas import DataFrame
#from pandas import Series
#from pandas import concat
from pandas import read_csv
#from pandas import datetime
import numpy as np

import glob
import pandas as pd
from operator import itemgetter
import itertools

In [2]:
#read future cloud QoS_provisioned data

files=glob.glob("csv_predictions/*.csv")
data=[]
providers_names=[]
for file in files:
    df=read_csv(file,header=0,names=['Av','Avg_rt','th'])
    providers_names.append(file)
    #df.drop(df.columns[[1,2]], axis=1, inplace=True)
    data.append(df)

In [3]:
services=[]
for sublist in data:
    services.append(sublist.values)
print(services[0])

[[0.8439863  0.34582695 0.337583  ]
 [0.8123784  0.3786761  0.37056246]
 [0.7852019  0.41068795 0.41737184]
 [0.7555617  0.44468617 0.44139966]]


In [4]:
#split 100 services into 3 groups
class_service_1=services[0:33]
class_service_2=services[33:66]
class_service_3=services[66:100]

In [5]:
### read end user future preferences weights for each service

In [6]:
user_weights_df=read_csv('future user_behavior.csv',header=0,names=['Av','Rt','Th'])

In [7]:
W=list(user_weights_df.values)


In [8]:
#test cases
#class_service_1=[[0.21,0.8,0.35],[0.021,0.5,0.12]]
#class_service_2=[[0.821,0.08,0.5],[0.901,0.4,0.15]]
#class_service_3=[[0.621,0.25,0.22], [0.321,0.08,0.05]]    #???
#W=[[0.8,0.6,0.6],[0.1,0.3,0.8],[0.621,0.25,0.22], [0.321,0.08,0.05]]          #???

In [9]:
#W[0][1]
len(class_service_2)

33

In [10]:
#--- COST FUNCTION ------------------------------------------------------------+
# function we are attempting to optimize (minimize) include the aggegator function

def func1(x):
    total_plan_value=0
    T_peroids=len(x)//3      #Time periods=4=total points or dimensions(x)/#classes which are 3 as assumed
 #   print("X=",x)
    for i in range(T_peroids):
        # assume the services are composed parallel aggragate QoS attributes values
        aggre_rt=max(class_service_1[x[3*i+0]][i,0],class_service_2[x[3*i+1]][i,0],class_service_3[x[3*i+2]][i,0])
        aggre_th=min(class_service_1[x[3*i+0]][i,1],class_service_2[x[3*i+1]][i,1],class_service_3[x[3*i+2]][i,1])
        aggre_cost=class_service_1[x[3*i+0]][i,2]+class_service_2[x[3*i+1]][i,2]+class_service_3[x[3*i+2]][i,2]
        value_plan_over_period_i=aggre_rt*W[i][0]+aggre_th*W[i][1]+aggre_cost*W[i][2]
        total_plan_value+=value_plan_over_period_i

    return total_plan_value

In [11]:


candidates_sols=[]
retrieved_sols=10
def add_sol(target_value,position):
    
    global candidates_sols
    candidates_sol=[]
    candidates_sols=sorted(candidates_sols, key=itemgetter(0))
    candidates_sols.reverse()
    
    #delete dubplicates
    candidates_sols=list(candidates_sols for candidates_sols,_ in itertools.groupby(candidates_sols))
    
    
    if len(candidates_sols)<retrieved_sols:
        candidates_sol.append(target_value)
        candidates_sol.extend(position)
        candidates_sols.append(candidates_sol)
        
        
    else:
        candidates_sol.append(target_value)
        candidates_sol.extend(position)
        candidates_sols[retrieved_sols-1]=candidates_sol
        

In [12]:
class Particle:
    def __init__(self,x0):
        self.position_i=[]                   # particle position
        self.velocity_i=[]                   # particle velocity
        self.pos_best_i=[]                   # best position individual
        self.local_best_fitness_value=-1     # best  individual fitness value
        self.fitness_value=-1                # current individual fitness value

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness using optimization function: costFunction
    def evaluate(self,costFunc):
        self.fitness_value=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.fitness_value > self.local_best_fitness_value or self.local_best_fitness_value==-1:
            self.pos_best_i=self.position_i
            self.local_best_fitness_value=self.fitness_value
            add_sol(self.fitness_value,self.position_i)

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.8      # constant inertia weight (how much to weigh the previous velocity)
        c1=2        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.randint(0,2)
            r2=random.randint(0,2)

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social
            

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]


            #want to make it integer number refers to services number (through bounds)==>update bounds
            #??? ok solved
            # adjust maximum position if necessary
            self.position_i[i]=int(self.position_i[i])
            if self.position_i[i]>bounds[i%3][1]:
                self.position_i[i]=bounds[i%3][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i%3][0]:
                self.position_i[i]=bounds[i%3][0]


In [13]:
pso_particles=[]

In [14]:
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        value_best_g=-1                   # best value for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,value_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].fitness_value > value_best_g or value_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    value_best_g=float(swarm[j].fitness_value)
                    #add_sol(value_best_g,pos_best_g)

            # cycle through swarm and update velocities and positioncostFunc
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        pos_best_g=np.array(pos_best_g)
        pos_best_g=pos_best_g.reshape(1,3)  #time period=4 , classes number=3
        print ('best global position:\n',pos_best_g)
        print ('\n \n best global target value:  ',value_best_g)
       
        #for j in range(0,num_particles):
            #print("\n \n position[",swarm[j].pos_best_i)
           # print("local_best_fitness_value[",swarm[j].local_best_fitness_value)
           

In [15]:
import random
import time
random.choice(range(len(class_service_1)))

#print(len(class_service_1)-1)

8

In [16]:
#initialize particle positision
T_periods=1 # T=1 or T=4
class_numbers=3
x0=[]
for i in range(T_periods):
    x0.append(random.choice(range(len(class_service_1))))
    x0.append(random.choice(range(len(class_service_2))))
    x0.append(random.choice(range(len(class_service_3))))

In [17]:
len(x0)

3

In [18]:
bounds=[(0, len(class_service_1)-1),(0,len(class_service_2)-1),(0,len(class_service_3)-1)]

In [19]:
bounds

[(0, 32), (0, 32), (0, 33)]

In [20]:
num_particle=10

In [21]:
#brute force service composition 
#for i in range (T)
total_plans=[]
for c_1 in range(len(class_service_1)):
    for c_2 in range(len(class_service_2)):
        for c_3 in range(len(class_service_3)):
            aggre_rt=max(class_service_1[c_1][0,0],class_service_2[c_2][0,0],class_service_3[c_3][0,0])
            aggre_th=min(class_service_1[c_1][0,1],class_service_2[c_2][0,1],class_service_3[c_3][0,1])
            aggre_cost=class_service_1[c_1][0,2]+class_service_2[c_2][0,2]+class_service_3[c_3][0,2]
            value_plan_over_period_0=aggre_rt*W[0][0]+aggre_th*W[0][1]+aggre_cost*W[0][2]
            total_plans.append([value_plan_over_period_0,c_1,c_2,c_3])

            

In [22]:

sorted_total_plans=sorted(total_plans, key=itemgetter(0))


In [23]:
sorted_total_plans.reverse()

brute force solutions

In [24]:
sorted_total_plans

[[2.835753094343131, 31, 25, 22],
 [2.825495630726731, 31, 25, 17],
 [2.823863696895931, 31, 25, 2],
 [2.814148430607931, 31, 25, 7],
 [2.8101618629319307, 31, 7, 22],
 [2.8080558882596627, 23, 25, 22],
 [2.807647792227331, 31, 25, 15],
 [2.8068622393683307, 31, 19, 22],
 [2.7999043993155306, 31, 7, 17],
 [2.7982724654847306, 31, 7, 2],
 [2.797798424643262, 23, 25, 17],
 [2.796604775751931, 31, 19, 17],
 [2.796166490812462, 23, 25, 2],
 [2.79526770699115, 15, 26, 22],
 [2.794972841921131, 31, 19, 2],
 [2.791252074143131, 31, 1, 22],
 [2.7885571991967306, 31, 7, 7],
 [2.787435928573531, 31, 26, 22],
 [2.786451224524462, 23, 25, 7],
 [2.785257575633131, 31, 19, 7],
 [2.7850102433747495, 15, 26, 17],
 [2.7833783095439495, 15, 26, 2],
 [2.7824646568484623, 23, 7, 22],
 [2.7820565608161307, 31, 7, 15],
 [2.7809946105267307, 31, 1, 17],
 [2.7799505861438623, 23, 25, 15],
 [2.77958797715083, 19, 26, 22],
 [2.779362676695931, 31, 1, 2],
 [2.7793203940570885, 8, 25, 22],
 [2.7791650332848628, 2

In [25]:
len(total_plans)
len(sorted_total_plans)

37026

In [26]:
relevant_info=10
stp10=sorted_total_plans[0:relevant_info]

In [27]:
tp10=total_plans[0:retrieved_sols]

relevant brute force sols

In [28]:
relevant_BF_sol=[list(x) for x in set(tuple(x) for x in tp10).intersection(set(tuple(x) for x in stp10))]

In [29]:
relevant_BF_sol

[]

In [30]:
len(relevant_BF_sol)

0

PSo solutions calculation

In [52]:
PSO(func1,x0,bounds,num_particles=10,maxiter=500)

FINAL:
best global position:
 [[31 25 22]]

 
 best global target value:   2.835753094343131


In [53]:
relevant_pso_sol=[list(x) for x in set(tuple(x) for x in candidates_sols).intersection(set(tuple(x) for x in stp10))]

In [54]:
relevant_pso_sol

[[2.823863696895931, 31, 25, 2],
 [2.835753094343131, 31, 25, 22],
 [2.8068622393683307, 31, 19, 22],
 [2.8101618629319307, 31, 7, 22],
 [2.814148430607931, 31, 25, 7],
 [2.825495630726731, 31, 25, 17],
 [2.807647792227331, 31, 25, 15],
 [2.7982724654847306, 31, 7, 2]]

In [55]:
len(relevant_pso_sol)

8

In [56]:
candidates_sols

[[2.835753094343131, 31, 25, 22],
 [2.825495630726731, 31, 25, 17],
 [2.823863696895931, 31, 25, 2],
 [2.814148430607931, 31, 25, 7],
 [2.8101618629319307, 31, 7, 22],
 [2.807647792227331, 31, 25, 15],
 [2.8068622393683307, 31, 19, 22],
 [2.7982724654847306, 31, 7, 2],
 [2.796604775751931, 31, 19, 17],
 [2.763146772027331, 31, 1, 15]]

In [57]:
recall=8/(8S+2)
print(recall)

0.8


In [58]:
precision=8/(8+2)
print(precision)

0.8
